---


# Dynamic Parking Pricing – Fully Annotated Walk-Through

This Jupyter-ready notebook is written in MyST Markdown so that you can open it **directly** in JupyterLab/VS Code (with *jupytext* enabled) or convert it to a classical `.ipynb` (``jupytext --to ipynb this_file.md``). Every code cell reproduces the logic you implemented, while the intervening Markdown cells explain *why* each step matters and how to avoid common pitfalls.  
If you simply double-click the file inside Colab it will import as a notebook automatically.

---

## 1  Environment & Package Installation

In real Kaggle/Colab sessions you normally pin exact versions to guarantee repeatability. Here we install the same minimal set used in the reference solution.

In [1]:
# Install core libraries silently ("-q") so the notebook looks tidy.
!pip install -q pathway haversine scikit-learn pandas numpy tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Why these libraries?
* **pandas/numpy** – canonical data wrangling duo.  
* **scikit-learn** – provides `MinMaxScaler` for demand normalisation.  
* **pathway** – streaming engine for real-time simulation.  
* **haversine** – one-liner distance computation between lat/lon pairs.

---

## 2  Dataset Upload & Initial Parsing

We load the CSV that holds raw sensor and metadata for each parking lot snapshot. The timestamp often arrives split across *date* and *time* strings, so we merge them before converting to `datetime64`.

In [2]:
from google.colab import files
import pandas as pd

uploaded = files.upload()  # choose your CSV (named e.g. parking.csv)
!mkdir -p /content/data
for fn in uploaded:
    !mv "$fn" /content/data/parking.csv

raw = pd.read_csv('/content/data/parking.csv')
raw.columns = raw.columns.str.strip()
raw['timestamp'] = pd.to_datetime(
    raw['LastUpdatedDate'] + ' ' + raw['LastUpdatedTime'], dayfirst=True
)
raw['day'] = raw['timestamp'].dt.date
raw.head()

Saving dataset.csv to dataset.csv


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,timestamp,day
0,0,BHMBCCMKT01,577,26.144536,91.736172,61,car,low,1,0,04-10-2016,07:59:00,2016-10-04 07:59:00,2016-10-04
1,1,BHMBCCMKT01,577,26.144536,91.736172,64,car,low,1,0,04-10-2016,08:25:00,2016-10-04 08:25:00,2016-10-04
2,2,BHMBCCMKT01,577,26.144536,91.736172,80,car,low,2,0,04-10-2016,08:59:00,2016-10-04 08:59:00,2016-10-04
3,3,BHMBCCMKT01,577,26.144536,91.736172,107,car,low,2,0,04-10-2016,09:32:00,2016-10-04 09:32:00,2016-10-04
4,4,BHMBCCMKT01,577,26.144536,91.736172,150,bike,low,2,0,04-10-2016,09:59:00,2016-10-04 09:59:00,2016-10-04


### Quality checks
Always eyeball missing-value patterns — a quick `raw.isna().mean()` surfaces nasty columns early.

---

## 3  Train–Test Split that Mimics Production

We pretend the **last 10 calendar days** are unseen future data. This chronological split avoids look-ahead bias present in random shuffles.

In [3]:
latest_days = sorted(raw['day'].unique())[-10:]
df_train = raw[~raw['day'].isin(latest_days)].copy()
df_test  = raw[ raw['day'].isin(latest_days)].copy()
print("Training range:", df_train['day'].min(), '→', df_train['day'].max())
print("Testing  range:", df_test['day'].min(),  '→', df_test['day'].max())

Training range: 2016-10-04 → 2016-12-09
Testing  range: 2016-12-10 → 2016-12-19


---

## 4  Feature Engineering

We transform raw counts into **ratios** so models generalise across lots of different capacity.

In [4]:
import numpy as np

for df in (df_train, df_test):
    df['occ_ratio']       = df['Occupancy'] / df['Capacity']
    df['queue_intensity'] = df['QueueLength'] / df['Capacity']
    df['TrafficCond']     = df['TrafficConditionNearby'].map({'low':0,'medium':1,'high':2})
    df['veh_type']        = df['VehicleType'].astype('category').cat.codes

**Pro-tip:** encode categoricals *after* concatenating train + test to keep category indices aligned.

---

## 5  Model ① – Baseline Linear Pricer

A transparent starting point: price nudges up in proportion to occupancy. We clamp the output to a safe band (
$5≤P≤20$) so you never double-charge.

In [5]:
class LinearPricer:
    def __init__(self, alpha=0.3, base=10):
        self.alpha, self.price0 = alpha, base
    def update(self, occ, cap):
        self.price0 += self.alpha * (occ/cap)
        return round(np.clip(self.price0, 5, 20), 2)

df_train['price_linear'] = [LinearPricer().update(r['Occupancy'], r['Capacity'])
                             for _, r in df_train.iterrows()]
df_train[['price_linear']].head()

,price_linear
0,10.03
1,10.03
2,10.04
3,10.06
4,10.08


---

## 6  Model ② – Demand-Based Pricing

We learn a *demand score* built from scaled features. The score then modulates a base $10$ currency unit.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

class DemandPricer:
    def __init__(self, lamb=0.8):
        self.scaler, self.lamb = MinMaxScaler(), lamb
        self.weights = np.array([0.4, 0.3, -0.1, 0.1, 0.1])  # Tunable
    def fit(self, df):
        self.scaler.fit(df[['occ_ratio','queue_intensity','TrafficCond','IsSpecialDay','veh_type']])
    def predict_row(self, row):
        x = self.scaler.transform([row])[0]
        demand = (x * self.weights).sum()
        price = 10 * (1 + self.lamb * demand)
        return round(np.clip(price, 5, 20), 2)

dpr = DemandPricer(); dpr.fit(df_train)
df_train['price_demand'] = [
    dpr.predict_row(r[['occ_ratio','queue_intensity','TrafficCond','IsSpecialDay','veh_type']])
    for _, r in df_train.iterrows()
]
df_train[['price_demand']].head()

### Interpreting the coefficients
Positive weights push price up with occupancy/queues; negative weights discount for heavy traffic that deters drivers.

---

## 7  Model ③ – Competitive Tweaks

We borrow the demand price and nudge it based on *nearest-neighbour* lots within 300 m. The logic mirrors supermarket fuel wars.

In [15]:
from haversine import haversine, Unit

class CompetitivePricer:
    def __init__(self, neighbour_table, lamb=0.8):
        self.base = DemandPricer(lamb)
        self.neighbours = neighbour_table  # DataFrame with lat,lon,current price
    def fit(self, df):
        self.base.fit(df)
    def predict_row(self, row):
      features = row[['occ_ratio', 'queue_intensity', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']]
      price = self.base.predict_row(features)  # only pass numeric features

      lat, lon = row['Latitude'], row['Longitude']
      occ_ratio = row['Occupancy'] / row['Capacity']

      comps = self.neighbours[
          self.neighbours.apply(
              lambda r: haversine((lat, lon), (r.Latitude, r.Longitude), Unit.METERS) < 300,
              axis=1
          )
      ]['CurrentPrice']

      for pc in comps:
          if occ_ratio > 0.95 and pc + 1 < price:
              price -= 0.5
          elif occ_ratio < 0.7 and pc - 1 > price:
              price += 0.5

      return round(np.clip(price, 5, 20), 2)

We build a *toy* competitor table so the cell runs without external data.

In [11]:
df_train['VehicleType'] = df_train['VehicleType'].astype('category').cat.codes
df_train['TrafficConditionNearby'] = df_train['TrafficConditionNearby'].map({'low': 0, 'medium': 1, 'high': 2})


In [ ]:
neigh = pd.DataFrame([
    {'Latitude': df_train.iloc[0]['Latitude']+0.001,
     'Longitude':df_train.iloc[0]['Longitude']+0.001,
     'CurrentPrice':10},
    {'Latitude': df_train.iloc[0]['Latitude']-0.001,
     'Longitude':df_train.iloc[0]['Longitude']-0.001,
     'CurrentPrice':12},
])

cpr = CompetitivePricer(neigh); cpr.fit(df_train)
features = ['occ_ratio', 'queue_intensity', 'TrafficConditionNearby', 'IsSpecialDay', 'VehicleType']
df_train['price_compet'] = [cpr.predict_row(r) for _, r in df_train.iterrows()]
df_train[['price_compet']].head()

---

## 8  Export Side-by-Side Comparison

Saving the merged output lets you score MAPE offline or pipe into a dashboard.

In [17]:
output_cols = ['SystemCodeNumber','timestamp','Occupancy','Capacity',
               'price_linear','price_demand','price_compet']

df_train[output_cols].to_csv('/content/output_prices.csv', index=False)
print('✅ CSV exported to /content/output_prices.csv')

✅ CSV exported to /content/output_prices.csv


---

## 9  Where to Go Next

1. Evaluate on **df_test** and compute classic error metrics.  
2. Wrap pricers inside a **Pathway** streaming graph for live deployment.  
3. Build an interactive **Bokeh** dashboard reading the `output_prices.csv` every 30 s.